Eliminar dos dados disciplinas que só são dadas num horario específico. => O.k.<br> 
Calcular, para cada disciplina, a média das notas e faltas dos alunos em todas as turmas.<br>
Calcular essas médias para cada horário. <br>
Fazer a diferença dessa média num horario com a media em todos os horarios. <br>

Dependendo dessa diferença ser positiva ou negativa, pode querer dizer que aquele horário tem um desempenho melhor ou pior.<br>
Poderia tirar o desvio padrão e fazer um z-score.

In [69]:
import pandas as pd

import numpy as np

data = pd.read_csv("input/turmas-plus-05.csv", encoding = 'utf-8')

In [70]:
data.head()

,id_turma,descricao_horario,id_componente_curricular,id_docente_interno,id_docente_externo,nivel_ensino,ano,periodo,noite34,noite12,...,REPROVADO POR MÉDIA E POR FALTAS,REPROVADO,CANCELADO,TRANCADO,REPROVADO POR NOTA,REPROVADO POR FALTAS,REPROVADO POR NOTA E FALTA,CUMPRIU,DESISTENCIA,TRANSFERIDO
0,57595659,5N123 (13/02/2017 - 01/07/2017),43535,5754625.0,NaN,GRADUAÇÃO,2017,1,True,True,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,57569222,5T6 4N4 5N12,26481,5756702.0,NaN,GRADUAÇÃO,2015,2,True,True,...,0.047619,0.047619,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1239378,2345M34 (23/03/2015 - 18/06/2015),56851,5752811.0,NaN,GRADUAÇÃO,2015,1,False,False,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,57563502,6M34,2041854,5753248.0,NaN,GRADUAÇÃO,2015,2,False,False,...,0.033333,0.000000,0.033333,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1232469,46M56,39997,5755795.0,NaN,GRADUAÇÃO,2014,2,False,False,...,0.043478,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [71]:
#Número de linhas
len(data.index)

#Valor das colunas
data.columns.values.tolist()


['id_turma',
 'descricao_horario',
 'id_componente_curricular',
 'id_docente_interno',
 'id_docente_externo',
 'nivel_ensino',
 'ano',
 'periodo',
 'noite34',
 'noite12',
 'tarde56',
 'tarde34',
 'tarde12',
 'manha56',
 'manha34',
 'manha12',
 'sabado',
 'aulas',
 'alunos',
 'mediana_faltas',
 'mediana_media_final',
 'APROVADO',
 'APROVADO POR NOTA',
 'REPROVADO POR MÉDIA E POR FALTAS',
 'REPROVADO',
 'CANCELADO',
 'TRANCADO',
 'REPROVADO POR NOTA',
 'REPROVADO POR FALTAS',
 'REPROVADO POR NOTA E FALTA',
 'CUMPRIU',
 'DESISTENCIA',
 'TRANSFERIDO']

In [131]:
"""
 Colunas de horários no arquivo turmas-plus-05.csv
 'noite34', -> 8
 'noite12',
 'tarde56',
 'tarde34',
 'tarde12',
 'manha56',
 'manha34',
 'manha12',
 'sabado', -> 16
"""

"\n Colunas de horários no arquivo turmas-plus-05.csv\n 'noite34', -> 8\n 'noite12',\n 'tarde56',\n 'tarde34',\n 'tarde12',\n 'manha56',\n 'manha34',\n 'manha12',\n 'sabado', -> 16\n "

In [72]:
#Remover turmas que contém aulas somente em um horário (Específico para o arquivo turmas-plus-05.csv ):
def removerTurmasCom1Horario(dataframe):
    for index, row in data.iterrows():
        count = 0
        #Dados das colunas 8 a 16
        for x in row[8:17]:
            if(x == True):
                count = count + 1
        if(count == 1):
            data.drop(index, inplace=True)
    dataframe.reset_index(drop = True)

removerTurmasCom1Horario(data)

data.head()

,id_turma,descricao_horario,id_componente_curricular,id_docente_interno,id_docente_externo,nivel_ensino,ano,periodo,noite34,noite12,...,REPROVADO POR MÉDIA E POR FALTAS,REPROVADO,CANCELADO,TRANCADO,REPROVADO POR NOTA,REPROVADO POR FALTAS,REPROVADO POR NOTA E FALTA,CUMPRIU,DESISTENCIA,TRANSFERIDO
0,57595659,5N123 (13/02/2017 - 01/07/2017),43535,5754625.0,NaN,GRADUAÇÃO,2017,1,True,True,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,57569222,5T6 4N4 5N12,26481,5756702.0,NaN,GRADUAÇÃO,2015,2,True,True,...,0.047619,0.047619,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,57578248,234567M123456 234567T123456 234567N12 (02/05...,55579,5755486.0,NaN,GRADUAÇÃO,2016,1,False,True,...,0.000000,0.081633,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,57575664,3M3456,43242,5752386.0,NaN,GRADUAÇÃO,2016,1,False,False,...,0.041667,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,57568614,6T2345,24689,5757016.0,NaN,GRADUAÇÃO,2015,2,False,False,...,0.000000,0.045455,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [73]:
##Cria novo DataFrame apenas com as colunas desejadas para o gráfico de barras
df_data = {'horarios': [ 'noite34', 'noite12', 'tarde56', 'tarde34', 'tarde12', 'manha56', 'manha34', 'manha12', 'sabado']}
df_horarios = pd.DataFrame(df_data)
df_horarios

,horarios
0,noite34
1,noite12
2,tarde56
3,tarde34
4,tarde12
5,manha56
6,manha34
7,manha12
8,sabado


In [74]:
#Cria nova tabela com alunos que passaram na matéria, independente do tipo de aprovação
data['PASSARAM_NA_MATERIA'] = data['APROVADO'] + data['APROVADO POR NOTA']
data.head()

,id_turma,descricao_horario,id_componente_curricular,id_docente_interno,id_docente_externo,nivel_ensino,ano,periodo,noite34,noite12,...,REPROVADO,CANCELADO,TRANCADO,REPROVADO POR NOTA,REPROVADO POR FALTAS,REPROVADO POR NOTA E FALTA,CUMPRIU,DESISTENCIA,TRANSFERIDO,PASSARAM_NA_MATERIA
0,57595659,5N123 (13/02/2017 - 01/07/2017),43535,5754625.0,NaN,GRADUAÇÃO,2017,1,True,True,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000
1,57569222,5T6 4N4 5N12,26481,5756702.0,NaN,GRADUAÇÃO,2015,2,True,True,...,0.047619,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.904762
5,57578248,234567M123456 234567T123456 234567N12 (02/05...,55579,5755486.0,NaN,GRADUAÇÃO,2016,1,False,True,...,0.081633,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.918367
6,57575664,3M3456,43242,5752386.0,NaN,GRADUAÇÃO,2016,1,False,False,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.958333
7,57568614,6T2345,24689,5757016.0,NaN,GRADUAÇÃO,2015,2,False,False,...,0.045455,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.954545


In [75]:
def mediaDeAprovados(dataFrame, tabela):
    lista_de_medias = []
    #Range específico para nosso dataframe
    for colIndex in range(8,17):
        #Faz a média da coluna, o resultado sai como um série pandas com 2 colunas: 
        #False -> Média de aprovados quando aquele horario não ocorre
        #True -> Média de aprovados para aquele horário
        media_ = dataFrame[tabela].groupby(dataFrame[dataFrame.columns[colIndex]]).mean()
        #Adiciona à lista o valor da segunda posição, que é oq nos interessa
        lista_de_medias.append(media_[1])
    return lista_de_medias

mediaAprovados = mediaDeAprovados(data, 'PASSARAM_NA_MATERIA')

df1 = pd.DataFrame({'media_aprovados': mediaAprovados})
df_horarios['media_aprovados'] = df1
df_horarios

,horarios,media_aprovados
0,noite34,0.870960
1,noite12,0.868998
2,tarde56,0.937137
3,tarde34,0.943798
4,tarde12,0.947888
5,manha56,0.965517
6,manha34,0.956311
7,manha12,0.963143
8,sabado,0.858883


In [76]:
from bokeh.charts import Bar, output_notebook, show

##Utilizando aqui PASSARAM_NA_MATERIA
p = Bar(df_horarios, 'horarios', values='media_aprovados', title="Média de aprovações por horário", 
        bar_width=0.6, legend=False)

output_notebook()

show(p)

Loading BokehJS ...

In [77]:
def mediaDeTrancamentos(dataFrame):
    lista_de_medias = []
    for colIndex in range(8,17):
        media_ = dataFrame['TRANCADO'].groupby(dataFrame[dataFrame.columns[colIndex]]).mean()
        lista_de_medias.append(media_[1])
    return lista_de_medias

mediaTrancamentos = mediaDeTrancamentos(data)

df2 = pd.DataFrame({'media_trancamentos': mediaTrancamentos})
df_horarios['media_trancamentos'] = df2
df_horarios

,horarios,media_aprovados,media_trancamentos
0,noite34,0.870960,0.001857
1,noite12,0.868998,0.001462
2,tarde56,0.937137,0.000364
3,tarde34,0.943798,0.000377
4,tarde12,0.947888,0.000307
5,manha56,0.965517,0.000268
6,manha34,0.956311,0.000215
7,manha12,0.963143,0.000287
8,sabado,0.858883,0.000343


In [78]:
from bokeh.embed import components
from bokeh.models import Axis
from bokeh.charts import Bar

p = Bar(df_horarios, 'horarios', values='media_trancamentos', title="Média de aprovações por horário", 
        bar_width=0.6, legend=False)

yaxis = p.select(dict(type=Axis, layout="left"))[0]
yaxis.formatter.use_scientific = False

output_notebook()

show(p)

Loading BokehJS ...

In [79]:
def mediaDeAlunos(dataFrame):
    lista_de_medias = []
    for colIndex in range(8,17):
        media_ = dataFrame['alunos'].groupby(dataFrame[dataFrame.columns[colIndex]]).mean()
        lista_de_medias.append(media_[1])
    return lista_de_medias

mediaAlunos = mediaDeAlunos(data)

df3 = pd.DataFrame({'media_alunos': mediaAlunos})
df_horarios['media_alunos'] = df3
df_horarios

,horarios,media_aprovados,media_trancamentos,media_alunos
0,noite34,0.870960,0.001857,24.086262
1,noite12,0.868998,0.001462,23.629179
2,tarde56,0.937137,0.000364,12.480349
3,tarde34,0.943798,0.000377,13.125884
4,tarde12,0.947888,0.000307,13.123028
5,manha56,0.965517,0.000268,10.920912
6,manha34,0.956311,0.000215,12.425966
7,manha12,0.963143,0.000287,11.889368
8,sabado,0.858883,0.000343,13.259259


In [80]:
p = Bar(df_horarios, 'horarios', values='media_alunos', title="Média do número de alunos por horário", 
        bar_width=0.6, legend=False)

yaxis = p.select(dict(type=Axis, layout="left"))[0]
yaxis.formatter.use_scientific = False

output_notebook()

show(p)

Loading BokehJS ...

In [ ]:
"""
Intervalos de aprovações considerados:
0.0|-0.3
0.3|-0.5
0.5|-0.7
0.8|-0.9
0.9|-
"""

### Adicionando nova coluna ao csv -> 'APROVACOES_INTERVALO'

In [90]:
#Preenche coluna APROVAÇÕES_INTERVALO no dataframe e cria um novo CSV
def preencherIntervalos(dataframe):
    
    #Cria nova tabela de aprovações por intervalo
    data["APROVACOES_INTERVALO"] = np.nan
    
    dataframe.loc[dataframe['APROVADO'] < 0.3, 'APROVACOES_INTERVALO'] = "0.0|-0.3"
    dataframe.ix[ np.logical_and(data.APROVADO>=0.3, data.APROVADO<0.5), ['APROVACOES_INTERVALO'] ] = "0.3|-0.5"
    dataframe.ix[ np.logical_and(data.APROVADO>=0.5, data.APROVADO<0.7), ['APROVACOES_INTERVALO'] ] = "0.5|-0.7"
    dataframe.ix[ np.logical_and(data.APROVADO>=0.7, data.APROVADO<0.8), ['APROVACOES_INTERVALO'] ] = "0.7|-0.8"
    dataframe.ix[ np.logical_and(data.APROVADO>=0.8, data.APROVADO<0.9), ['APROVACOES_INTERVALO'] ] = "0.8|-0.9"
    dataframe.loc[dataframe['APROVADO'] > 0.9, 'APROVACOES_INTERVALO'] = "0.9|-"
    
    dataframe.to_csv('input/turmas-att05.csv', index=False)

preencherIntervalos(data)

##Verificando resultado
df1 = data[['APROVADO','APROVACOES_INTERVALO']]
df1

,APROVADO,APROVACOES_INTERVALO
0,0.956522,0.9|-
1,0.666667,0.5|-0.7
5,0.918367,0.9|-
6,0.875000,0.8|-0.9
7,0.954545,0.9|-
8,0.875000,0.8|-0.9
10,0.181818,0.0|-0.3
11,0.933333,0.9|-
12,1.000000,0.9|-
13,0.500000,0.5|-0.7
